In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sjoert.stellar
import pandas as pd
import os
from tqdm import tqdm
from cleaning_functions import q_cuts,field_check,filter_split,flux_unc_val
from astropy import coordinates as coord
import json
from numpyencoder import NumpyEncoder

sjoertpath = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\Sjoert_Flares'

In [13]:
# # Code to remove all clean files and clean logs from the sjoert_flares directory.
def remove_clean():
    for folder in os.listdir(sjoertpath)[::-1]:
        if 'ZTF' in folder:
            folderpath = os.path.join(sjoertpath,folder)
            for file in os.listdir(folderpath):
                if 'clean' in file:
                    os.remove(os.path.join(folderpath,file))
# remove_clean()

In [8]:
def clean_data(datapath,ZTF_ID,savepath=None,verbose=False):
    """Cleans ZTF batch request data using the qcuts function. The removed data is neatly logged on a per filter - per field basis. The cleaning
    now works in such a way that only the data from the primary field (defined as the field in which most measurements were made) is used.
    The uncertainty on the flux measurements is validated and subsequently updated following the method of the ZFPS user guide: 
    https://web.ipac.caltech.edu/staff/fmasci/ztf/zfps_userguide.pdf. The cleaned data contains: time in jd, forced difference image PSF-fit flux [DN],
      the 1-sigma uncertainty in the forced difference image PSF-fit flux [DN], the photometric zeropoint for difference image [mag] and the filter 
      (one of ZTF_g, ZTF_r and ZTF_i). The cleaning log contains, for every field in every filter, whether there are even viable measurements and if
      there are if the field in question is the primary field, what the median zeropoint is, what the standard deviation of the zeropoints is, how
      many of the data points were removed in cleaning and the median chi-square of the datapoints before cleaning. For every filter the median 
      chi-square after cleaning is saved only for the primary field - this will differ only slightly from the median chi-square before cleaning. 
      Both cleaned data and cleaning log are saved as json files in the form "(ZTF_ID)_clean_data.json" and "(ZTF_ID)_clean_log.json".
      
      IMPORTANT: dependencies are the numpy, pandas, json and os packages as well as a special json NumpyEncoder by Hunter M. Allen (https://pypi.org/project/numpyencoder/).

    Args:
        datapath (str): Path to the raw data in the form path\to\data\batchf_reqxxxxxxxxxxxxxxxxx_lc.txt.
        ZTF_ID (str): ZTF identifier of the transient.
        savepath (str, optional): Path to folder in which clean data and cleaning log will be saved. Defaults to None, in which case data is printed if verbose is True, otherwise it is lost.
        verbose (bool, optional): Controls the (amount of) print statements in the function. Defaults to False.
    """
    #Read in the raw data from the data path as a Pandas DataFrame. 
    columns = ['sindex', 'field', 'ccdid', 'qid', 'filter', 'pid', 'infobitssci', 'sciinpseeing', 'scibckgnd', 'scisigpix', 'zpmaginpsci', 'zpmaginpsciunc', 'zpmaginpscirms', 'clrcoeff', 'clrcoeffunc', 'ncalmatches', 'exptime', 'adpctdif1', 'adpctdif2', 'diffmaglim', 'zpdiff', 'programid', 'jd', 'rfid', 'forcediffimflux', 'forcediffimfluxunc', 'forcediffimsnr', 'forcediffimchisq', 'forcediffimfluxap', 'forcediffimfluxuncap', 'forcediffimsnrap', 'aperturecorr', 'dnearestrefsrc', 'nearestrefmag', 'nearestrefmagunc', 'nearestrefchi', 'nearestrefsharp', 'refjdstart', 'refjdend', 'procstatus']
    dtypes = [(columns[x],float) for x in range(len(columns))]
    dtypes[4] = ('filter',r'U8')
    data = pd.DataFrame(np.genfromtxt(datapath,skip_header=53,dtype=dtypes))

    clean_data_full = pd.DataFrame() #an empty frame on which the data from every filter will be vertically stacked.
    iok = q_cuts(data) #very first quality check, mask array of good data points.
    data_ok = data[iok]
    logdict = {} #dictionary that will form the log.json file

    filters = np.unique(data['filter'])
    filtermasks = [data['filter'] == f for f in filters]
    # fields,field_counts = np.unique(data['field'],return_counts=True) #return_counts for picking the primary field
    fields,_ = np.unique(data_ok['field'],return_counts=True) #Do we want the primary field on data or on data_ok?
    fieldmasks = [data['field'] == fid for fid in fields]

    if iok.sum() == 0: #this might occur, this prevents an error
        print(f"{ZTF_ID}: no viable data found in the batch request. Proceeding to next file.")
        return 
    
    for i,filter in enumerate(filters):
            logdict[filter] = {}
            logdict[filter]["no_viable_data"] = 0 #can be used for a check when loading in the data; if this is True then the data is useless in this particular filter 
            filtermask = filtermasks[i]
            iok_filter = (iok * filtermask) #this checks if something is ok according to qcuts and is in a certain filter. Has the same len as data.
            
            if iok_filter.sum() == 0: #this might occur, this prevents an error
                print(f"{ZTF_ID}: no viable data found in {filter}. Proceeding to next filter.")
                logdict[filter]["no_viable_data"] = 1
                continue

            #If we take the primary field on a per filter basis use three lines below.
            data_ok_filter = data[iok_filter]
            filter_field_counts = [np.sum(data_ok_filter['field'] == fid) for fid in fields] #count for each field we know to have in the uncleaned data how often it appears in this filter. Might yield 0's! 
            primary_field = [c == np.max(filter_field_counts) for c in filter_field_counts] #should be this one if we want to pick the primary on a per filter basis

            for j,fid in enumerate(fields):
                field_mask = fieldmasks[j]
                iok_filter_field = iok_filter * field_mask #this checks if something is ok according to qcuts, is in a certain filter and is in a certain field. Has the same len as data.

                logdict[filter][fid] = {}
                if iok_filter_field.sum() == 0: #this might occur, this prevents an error
                    print(f"{ZTF_ID}: no viable data found in field {fid} of filter {filter}. Proceeding to next field for this filter.")
                    logdict[filter][fid]["no_viable_data"] = 1 #can be used for a check when loading in the data; if this is True then the data is useless in this particular field / filter combo
                    continue

                data_ok_filter_field = data[iok_filter_field]
                data_filter_field = data[filtermask*field_mask] #this is the uncleaned data of this field in this filter
                zeropoint = data_ok_filter_field['zpdiff'].values

                logdict[filter][fid] = {"primary_field":int(primary_field[j]),
                                        "median_zeropoint":np.median(zeropoint),'std_zeropoint':np.std(zeropoint),
                                        "removed_in_cleaning":np.sum(np.invert(iok_filter_field)),
                                        "amount_before_cleaning": len(iok_filter_field),
                                        "median_chi2":np.median(data_filter_field['forcediffimchisq']),
                                            "no_viable_data":0}

                if primary_field[j]:
                    #correct the errors of the clean data in this filter (only on the primary field)
                    new_unc = np.array(flux_unc_val(data_ok_filter_field))
                    #the median chi squared after is that of the good data in the primary field of the respective filter
                    logdict[filter]["median_chi2_after"] = np.median(data_ok_filter_field['forcediffimchisq']) 
                    clean_data_filt = pd.DataFrame({'time':data_ok_filter_field['jd'],'flux':data_ok_filter_field['forcediffimflux'],
                                                   'flux_unc':new_unc,'zeropoint':data_ok_filter_field['zpdiff'],
                                                   'filter':data_ok_filter_field['filter']})
                    clean_data_full = pd.concat([clean_data_full,clean_data_filt],ignore_index=True)
        
    if savepath != None:
        # clean_data_full.to_json(os.path.join(savepath,str(ZTF_ID)+'_clean_data.json'))
        clean_data_full.to_csv(os.path.join(savepath,str(ZTF_ID)+'_clean_data.txt'),sep='\t',index=None,header=None,mode='a')
        with open(os.path.join(savepath,str(ZTF_ID)+'_clean_log.json'),'w') as outfile:
            json.dump(logdict,outfile,indent=4,ensure_ascii=False,separators=(',',':'),cls=NumpyEncoder)
    else:
        print('No savepath provided. Dumping results, shown if verbose set to True.')
        if verbose:
            print(clean_data_full.to_markdown())
            print()
            print(logdict)
                    


In [14]:
def clean_iter():
    for folder in os.listdir(sjoertpath)[::-1]:
        if "ZTF" in folder:
            folderpath = os.path.join(sjoertpath,folder)
            ztf_id = os.path.split(folderpath)[-1]
            for file in os.listdir(folderpath):
                if 'clean' not in file and 'parameters' not in file:
                    filepath = os.path.join(folderpath,file)
                    try:
                        clean_data(filepath,ztf_id,savepath=folderpath)
                    except ValueError as err:
                        print(err)
                        print(folderpath,ztf_id)
                        return
                    
# clean_iter()     
                


ZTF20aagkqky: no viable data found in field 1869.0 of filter ZTF_i. Proceeding to next field for this filter.
ZTF20aabhraq: no viable data found in field 1510.0 of filter ZTF_i. Proceeding to next field for this filter.
ZTF20aabhraq: no viable data found in field 1511.0 of filter ZTF_i. Proceeding to next field for this filter.
ZTF20aabhraq: no viable data found in field 1510.0 of filter ZTF_r. Proceeding to next field for this filter.
ZTF19adcddzk: no viable data found in field 1749.0 of filter ZTF_i. Proceeding to next field for this filter.
ZTF19adawqog: no viable data found in field 1711.0 of filter ZTF_i. Proceeding to next field for this filter.
ZTF19accdntg: no viable data found in field 1593.0 of filter ZTF_i. Proceeding to next field for this filter.
ZTF19acaakoh: no viable data found in field 1831.0 of filter ZTF_i. Proceeding to next field for this filter.
ZTF19abzrhgq: no viable data found in field 1347.0 of filter ZTF_i. Proceeding to next field for this filter.
ZTF19abxwb

### Testing adding a header after the fact

In [14]:
#just a random file
testpath = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\Sjoert_Flares\ZTF18abvmqur\ZTF18abvmqur_clean_data.txt'

def make_header(logfilepath):
    logfilename = os.path.split(logfilepath)[-1]
    logstr = f"#See {logfilename} for thorough overview of cleaning process.\n#Headers: time [jd], flux [ujy], flux uncertainty [ujy], zeropoint [mag], filter"
    with open(logfilepath,'r') as logfile:
        logdata = json.load(logfile)
    filters = ["ZTF_g","ZTF_r","ZTF_i"]
    for filt in filters:
        try:
            filter_data = logdata[filt]
        except KeyError:
            continue
        if filter_data['no_viable_data']:
            if filt =='ZTF_g' or filt == "ZTF_r":
                logstr += f'No viable data in {filt}. This data will thus not be fitted.\n#'
            else:
                logstr += f'No viable data in {filt}.\n#'
    logstr += '\n'
    return logstr

def line_prepender(filename):
    #filename must be the absolute path to the clean data
    filedir,ztf_clean_data = os.path.split(filename)
    print(ztf_clean_data)
    logdir = os.path.join(filedir,ztf_clean_data.split("_")[0]+"_clean_log.json")
    headerstring = make_header(logdir)
    with open(filename, 'r+') as f:
        content = f.read()
        if '#See' in content:
            return
        f.seek(0, 0)
        f.write(headerstring.rstrip('\r\n') + '\n' + content)


# make_header(r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\Sjoert_Flares\ZTF18abvmqur\ZTF18abvmqur_clean_log.json')
def prepend_sjoert_files():
    for folder in os.listdir(sjoertpath)[::-1]:
            if "ZTF" in folder:
                print(folder)
                folderpath = os.path.join(sjoertpath,folder)
                ztf_id = os.path.split(folderpath)[-1]
                for file in os.listdir(folderpath):
                    if "clean_data.txt" in file:        
                        filepath = os.path.join(folderpath,file)
                        line_prepender(filepath)
                        


In [16]:
def clean_iter(datapath):
    for folder in os.listdir(datapath)[::-1]:
        if folder.isnumeric(): #only the once with years as names
            for ZTF_folder in tqdm(os.listdir(os.path.join(datapath,folder))):
                if "ZTF" in ZTF_folder:
                    folderpath = os.path.join(datapath,folder,ZTF_folder)
                    ztf_id = os.path.split(folderpath)[-1]
                    for file in os.listdir(folderpath):
                        if 'clean' not in file and 'parameters' not in file:
                            filepath = os.path.join(folderpath,file)
                            print(filepath)
                            try:
                                clean_data(filepath,ztf_id,savepath=folderpath,verbose=False)
                                line_prepender(os.path.join(folderpath,f'{ZTF_folder}_clean_data.txt'))
                            except ValueError as err:
                                print(err)
                                print(folderpath,ztf_id)
                                return

clean_iter(r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data')

  0%|          | 0/86 [00:00<?, ?it/s]

C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\23\ZTF23aaaaiuc\batchfp_req0000369051_lc.txt
ZTF23aaaaiuc: no viable data found in field 1773.0 of filter ZTF_g. Proceeding to next field for this filter.
ZTF23aaaaiuc: no viable data found in field 772.0 of filter ZTF_i. Proceeding to next field for this filter.
ZTF23aaaaiuc: no viable data found in field 1773.0 of filter ZTF_r. Proceeding to next field for this filter.
ZTF23aaaaiuc_clean_data.txt


In [13]:
import os 
datapath = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data'
for folder in os.listdir(datapath):
    if folder.isnumeric():
        for ZTF in os.listdir(os.path.join(datapath,folder)):
            for f in os.listdir(os.path.join(datapath,folder,ZTF)):
                print(f)

batchfp_req0000358734_lc.txt
batchfp_req0000358735_lc.txt
batchfp_req0000358736_lc.txt
batchfp_req0000358737_lc.txt
batchfp_req0000358738_lc.txt
batchfp_req0000358739_lc.txt
batchfp_req0000358740_lc.txt
batchfp_req0000358741_lc.txt
batchfp_req0000358742_lc.txt
batchfp_req0000358743_lc.txt
batchfp_req0000358744_lc.txt
batchfp_req0000358745_lc.txt
batchfp_req0000358746_lc.txt
batchfp_req0000358747_lc.txt
batchfp_req0000358748_lc.txt
batchfp_req0000358749_lc.txt
batchfp_req0000358750_lc.txt
batchfp_req0000358751_lc.txt
batchfp_req0000358752_lc.txt
batchfp_req0000358753_lc.txt
batchfp_req0000358754_lc.txt
batchfp_req0000358755_lc.txt
batchfp_req0000358756_lc.txt
batchfp_req0000358757_lc.txt
batchfp_req0000358758_lc.txt
batchfp_req0000358759_lc.txt
batchfp_req0000358760_lc.txt
batchfp_req0000358761_lc.txt
batchfp_req0000358762_lc.txt
batchfp_req0000358763_lc.txt
batchfp_req0000358764_lc.txt
batchfp_req0000358765_lc.txt
batchfp_req0000358766_lc.txt
batchfp_req0000358767_lc.txt
batchfp_req000

In [17]:
path = 'C:\\Users\\timvd\\Documents\\Uni 2023-2024\\First Research Project\\Data\\22\\ZTF22aartgte'
datapath = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\22\ZTF22aartgte\batchfp_req0000369699_lc.txt'
clean_data(datapath,'ZTF22aartgte',savepath=path,verbose=True)

ZTF22aartgte: no viable data found in the batch request. Proceeding to next file.


In [28]:
datapath = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data'
s = 0
no_clean_file = open(os.path.join(datapath,'no_clean_data.txt'),'w')
for folder in os.listdir(datapath)[::-1]:
    if folder.isnumeric(): #only the once with years as names
        for ZTF_folder in os.listdir(os.path.join(datapath,folder)):
            if "ZTF" in ZTF_folder:
                folderpath = os.path.join(datapath,folder,ZTF_folder)
                ztf_id = os.path.split(folderpath)[-1]
                if f"{ZTF_folder}_clean_data.txt" not in os.listdir(folderpath):
                    # print(ZTF_folder,to=no_clean_file)
                    no_clean_file.write(f"{ZTF_folder}\n")
                    s+=1
no_clean_file.close()
print(s)

65
